In [4]:
import os
import glob

import pandas as pd

In [5]:
DATA_DIR = '../data'
WEIGHT_MIN = 1
WEIGHT_MAX = 10

In [6]:
def get_weights(group):
    min_count = group['Count'].min()
    max_count = group['Count'].max()
    group['Weight'] = WEIGHT_MIN + (group['Count'] - min_count) * (WEIGHT_MAX - 1) / (max_count - min_count)
    return group

In [7]:
all_dfs = []

csv_files = glob.glob(
    os.path.join(f"{DATA_DIR}/tags", '*.csv')
)
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    all_dfs.append(df)
    
combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df = combined_df.groupby('Tag', as_index=False)['Count'].sum()
combined_df = combined_df.sort_values(by='Count', ascending=False)

combined_df[['Key', 'Value']] = combined_df['Tag'].str.extract(r"\('([^']*)', '([^']*)'\)")

combined_df = combined_df[
    combined_df['Key'].isin([
        'highway', 
        'surface',
        'footway',
        'surface',
        'oneway',
        'lit',
        'crossing',
    ])
]

combined_df = combined_df.groupby('Key').apply(get_weights)

output_dir = f"{DATA_DIR}/weights"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

combined_df.to_csv(f"{output_dir}/weights.csv", index=False)

/var/folders/5n/flhvrbg930x4c6qpncw2qq_80000gn/T/ipykernel_42730/3189712212.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined_df = combined_df.groupby('Key').apply(get_weights)
